<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Conv1D, Dense, AveragePooling1D, Flatten
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(test.columns)-1, 1), kernel_size=5, filters=12))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(activation="sigmoid", kernel_size=5, filters=6))
model.add(AveragePooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 66, 12)            72        
                                                                 
 average_pooling1d (AverageP  (None, 33, 12)           0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 29, 6)             366       
                                                                 
 average_pooling1d_1 (Averag  (None, 14, 6)            0         
 ePooling1D)                                                     
                                                                 
 flatten (Flatten)           (None, 84)                0         
                                                                 
 dense (Dense)               (None, 1)                 8

In [6]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 17s 3ms/step - loss: 3.3352e-04 - accuracy: 0.9999


In [7]:
test_y = test.pop('Label')
test_x = test

In [8]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [9]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

79666/79666 [==============================] - 126s 2ms/step


In [10]:
confusion_matrix(test_y, pred_y)

array([[1255469,     272],
       [     12, 1293535]])

In [11]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.99999   0.99978   0.99989   1255741
           1    0.99979   0.99999   0.99989   1293547

    accuracy                        0.99989   2549288
   macro avg    0.99989   0.99989   0.99989   2549288
weighted avg    0.99989   0.99989   0.99989   2549288

